In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from dash import dash_table
from datetime import date

from col_consulta import *
from col_credentials import *

In [3]:
df=consulta_mae('Bogotá DC','SERVICIO NACIONAL DE APRENDIZAJE SENA','2018-01-01','2019-12-31','0','100000000000',None)
df

,id_contrato,id_proceso,valor_contrato,entidad,departamento_entidad,orden,modalidad,tipo_contrato,codigo_unspsc,objeto_contrato,descripcion_proceso,nombre_proveedor,tipo_documento_proveedor,numero_documento_proveedor,fecha_inicio,fecha_fin,fecha_firma,url,fuente
0,12,19-12-9464395-8613240,21502371.0,SERVICIO NACIONAL DE APRENDIZAJE SENA,Bogotá DC,NACIONAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",PRESTACIÓN DE LOS SERVICIOS PERSONALES DE CARÁ...,ARGIRO DE JESUS VERGARA MUÑOZ,Nit de Persona Natural,98571173,2019-01-18T00:00:00.000,2019-12-24T00:00:00.000,2019-01-17T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
1,2151,18-13-8339555-7647458,34155000.0,SERVICIO NACIONAL DE APRENDIZAJE SENA,Bogotá DC,NACIONAL,Contratación Mínima Cuantía,Compraventa,531116,"Ropa, Maletas y Productos de Aseo Personal",Adquisición de dotación calzado caballero para...,DEPORTIVOS CARVAJAL SAS,Nit de Persona Jurídica,900965542,2018-08-28T00:00:00.000,2018-12-30T00:00:00.000,2018-08-27T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
2,RV1332018,18-12-8692588-7903196,3600000.0,SERVICIO NACIONAL DE APRENDIZAJE SENA,Bogotá DC,NACIONAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",Prestar los servicios personales como Instruct...,SILVIA CAROLA OROZCO PEÑA,Cédula de Ciudadanía,36289485,2018-11-16T00:00:00.000,2018-12-16T00:00:00.000,2018-11-16T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
3,906,18-12-8362359-7606725,25008137.0,SERVICIO NACIONAL DE APRENDIZAJE SENA,Bogotá DC,NACIONAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,861018,Servicios Educativos y de Formación,PRESTAR CON PLENA AUTONOMIA TECNICA Y ADMINIST...,ROCA SANCHEZ CARLOS,Cédula de Ciudadanía,93424113,2018-02-01T00:00:00.000,2018-08-30T00:00:00.000,2018-01-22T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
4,790,18-12-7847857-7123988,21456000.0,SERVICIO NACIONAL DE APRENDIZAJE SENA,Bogotá DC,NACIONAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",Prestar los servicios profesionales de carácte...,SERGIO EDMUNDO BARAJAS ACEVEDO,Cédula de Ciudadanía,13926481,2018-02-01T00:00:00.000,2018-10-02T00:00:00.000,2018-01-24T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75444,0699,18-13-8043350-7359366,2486866.0,SERVICIO NACIONAL DE APRENDIZAJE SENA,Bogotá DC,NACIONAL,Contratación Mínima Cuantía,Compraventa,422615,"Equipo Médico, Accesorios y Suministros",Compra de materiales de formación en el área a...,DORADO HECTOR CENTRO AGROPECUARIO DE NARIÑO,Cédula de Ciudadanía,5248726,2018-05-02T00:00:00.000,2018-05-17T00:00:00.000,2018-04-27T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
75445,DR025818,18-12-8038572-7303031,18201867.0,SERVICIO NACIONAL DE APRENDIZAJE SENA,Bogotá DC,NACIONAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",Prestar los servicios personales para apoyar e...,ELKIN ANDRES PARRA GIRALDO,Cédula de Ciudadanía,1094932217,2018-01-19T00:00:00.000,2018-12-27T00:00:00.000,2018-01-18T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
75446,410186,19-12-8980199-8169193,33920000.0,SERVICIO NACIONAL DE APRENDIZAJE SENA,Bogotá DC,NACIONAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",Prestación de servicios profesionales o de apo...,EDDINSON ORTEGA MARTINEZ,Cédula de Ciudadanía,12198097,2019-02-04T00:00:00.000,2019-12-04T00:00:00.000,2019-02-01T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
75447,1020,18-13-8192877-7565205,15170070.0,SER

In [4]:
def consulta_MAE_simple():
    clas_unspsc=pd.read_csv('datos/clasificador_de_bienes_y_servicios.csv')
    clas_unspsc=clas_unspsc[['Nombre Clase','Código Clase']].drop_duplicates()
    clas_unspsc.set_index('Código Clase',inplace=True)
    app=JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
    df=pd.read_csv('datos/contratos_entidad.csv')
    app.layout=html.Div([
        html.H1("Análisis de datos del sistema de compra pública"),
        html.H2("Datos de la compra pública"),
        html.H3("Seleccione el departamento"),
        dcc.Dropdown(
            id='departamento',
            options=[{'label': i, 'value': i} for i in dptos],
            multi=True,
            value='Cundinamarca'),
        html.H3("Seleccione el año"),
        dcc.Dropdown(
            id='anio',
            options=[{'label': i, 'value': i} for i in ['2018','2019','2020','2021','2022']],
            multi=True,
            value='2022'),
        html.H3("Seleccione la entidad"),
        dcc.Dropdown(
            id='dropdown_entidad',
            multi=True,
            value='Todas',
            ),
        
        dbc.Row([
            
            dbc.Col([
            html.Div(id='card_1'),
            html.Div(id='card_2'),
            html.Div(id='card_3'),
            ]),
            dbc.Col([
            dcc.Graph(id='grafica_1'),
            ]),
            dbc.Col([
            dcc.Graph(id='grafica_2'),
            ]),    

    ])
        
    ])
    @app.callback(Output('dropdown_entidad','options'),
                [Input('departamento','value'),
                Input('anio','value'),])
    def update_dropdown_entidad(departamento,anio):
        if departamento is None:
            departamento=dptos
        if anio is None:
            anio=2022
        if type(departamento)==str:
            departamento=[departamento]
        if type(anio)==str:
            anio=[anio]
        lista_consulta=[]
        for dpto in departamento:
            for a in anio:
                a=int(a)
                df_dpto=consulta_datos_departamento(dpto,a)
                lista_consulta.append(df_dpto)
        df=pd.concat(lista_consulta)
        return [{'label': i, 'value': i} for i in list(df['entidad'].unique())+['Todas']]


    @app.callback(
        Output('card_1', 'children'),
        Output('card_2', 'children'),
        Output('card_3', 'children'),
        Output('grafica_1', 'figure'),
        Output('grafica_2', 'figure'),
        Input('departamento', 'value'),
        Input('anio', 'value'),
        Input('dropdown_entidad', 'value'),
        )
    def update_output(departamento,anio,entidad):
        if type(departamento)==str:
            departamento=[departamento]
        if type(anio)==str:
            anio=[anio]
        lista_consulta=[]
        for dpto in departamento:
            for an in anio:
                an=int(an)
                df_temp=consulta_datos_departamento(dpto,an)
                lista_consulta.append(df_temp)
        df=pd.concat(lista_consulta)
        if type(entidad)==str and entidad!='Todas':
            entidad=[entidad]
            df=df[df['entidad'].isin(entidad)]
        if type(entidad)==str and entidad=='Todas':
            None
        else:
            df=df[df['entidad'].isin(entidad)]
        df=df.dropna()
        df=df.drop_duplicates()
        table=dash_table.DataTable(
            id='table',
            columns=[{"name": i, "id": i} for i in df.columns],
            data=df.to_dict('records'),
            style_cell={'textAlign': 'left'},
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
        )
        card_1=dbc.Card(
            dbc.CardBody(
                [
                    html.H4("Número de contratos", className="card-title"),
                    html.P(
                        df['cantidad'].sum(),
                        className="card-text",
                    ),
                ]
            ),
            style={"width": "18rem"},
        )
        card_2=dbc.Card(
            dbc.CardBody(
                [
                    html.H4("Valor total de contratos", className="card-title"),
                    html.P(
                        df['suma_valor_contrato'].sum(),
                        className="card-text",
                    ),
                ]
            ),
            style={"width": "18rem"},
        )
        card_3=dbc.Card(
            dbc.CardBody(
                [
                    html.H4("Promedio valor de Contratos", className="card-title"),
                    html.P(
                        df['suma_valor_contrato'].sum()/df['cantidad'].sum(),
                        className="card-text",
                    ),
                ]
            ),
            style={"width": "18rem"},
        ),
        fig1=px.bar(df, x="año", y="suma_valor_contrato", color="departamento_entidad")
        fig2=px.bar(df, x="año", y="cantidad",  color="departamento_entidad")
        return card_1,card_2,card_3,fig1,fig2#,table

    return app.run_server(mode='inline',debug=True)

In [5]:
def consulta_MAE(port=np.random.randint(1000,9999)):
    app=JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
    clas_unspsc=pd.read_csv('datos/clasificador_de_bienes_y_servicios.csv')
    clas_unspsc=clas_unspsc[['Nombre Clase','Código Clase']].drop_duplicates()
    clas_unspsc.set_index('Código Clase',inplace=True)
    df=pd.read_csv('datos/contratos_entidad.csv')
    app.layout=html.Div([
        html.H1("Análisis de datos del sistema de compra pública"),
        html.H2("Datos de la compra pública"),
        dbc.Row([            
            dbc.Col([
        html.H3("Seleccione el departamento"),
        dcc.Dropdown(
            id='departamento',
            options=[{'label': i, 'value': i} for i in dptos],
            multi=True,
            value='Cundinamarca'),
            ]),         
            dbc.Col([
        html.H3("Seleccione el valor inicial del rango"),
        dcc.Input(
            id='valor_inicial',
            type='number',
            value=0),
            ]),
        dbc.Col([
        html.H3("Seleccione el valor final del rango"),
        dcc.Input(
            id='valor_final',
            type='number',
            value=1000000000),

            ]),
        ])    ,
        dbc.Row([            
            dbc.Col([
        html.H3("Seleccione rango de fechas"),
        dcc.DatePickerRange(
        id='my-date-picker-range',
        min_date_allowed=date(2000, 1, 1),
        max_date_allowed=date(2023, 1, 1),
        start_date=date(2022,1, 1),
        end_date=date(2022, 9, 30)
    )

            ]),
            dbc.Col([
        html.H3("Seleccione Clase UNSPSC"),
        dcc.Dropdown(
            id='unspsc',
            multi=True,
            options=[{'label': str(i)+' - '+clas_unspsc['Nombre Clase'][i], 'value': i} for i in clas_unspsc.index],
            ),
            ]),
    ]),
        dbc.Row([
                dbc.Col([
        html.H3("Seleccione la entidad"),
        dcc.Dropdown(
            id='dropdown_entidad',
            multi=True,
            value='Todas',
            ),
            ]),
               ]),
        dbc.Row([
            
            dbc.Col([
            html.Div(id='card_1'),
            html.Div(id='card_2'),
            html.Div(id='card_3'),
            ]),
            dbc.Col([
            dcc.Graph(id='grafica_1'),
            ]),
            dbc.Col([
            dcc.Graph(id='grafica_2'),
            ]),    

    ])
        
    ])
    @app.callback(Output('dropdown_entidad','options'),
                [Input('departamento','value'),])
    def update_dropdown_entidad(departamento):
        anio=[str(i) for i in range(2000,2023)]
        if departamento is None:
            departamento=dptos
        if type(departamento)==str:
            departamento=[departamento]
        lista_consulta=[]
        for dpto in departamento:
            for a in anio:
                a=int(a)
                df_dpto=consulta_datos_departamento(dpto,a)
                lista_consulta.append(df_dpto)
        df=pd.concat(lista_consulta)
        return [{'label': i, 'value': i} for i in list(df['entidad'].unique())+['Todas']]


    @app.callback(
        Output('card_1', 'children'),
        Output('card_2', 'children'),
        Output('card_3', 'children'),
        Output('grafica_1', 'figure'),
        Output('grafica_2', 'figure'),
        Input('departamento', 'value'),
        Input('anio', 'value'),
        Input('dropdown_entidad', 'value'),
        Input('valor_inicial', 'value'),
        Input('valor_final', 'value'),
        Input('unspsc', 'value'),
        Input('my-date-picker-range', 'start_date'),
        Input('my-date-picker-range', 'end_date'),
        )
    def update_output(departamento,anio,entidad,unspsc,valor_inicial,valor_final,fecha_inicio,fecha_fin):
        valor_inicial=str(valor_inicial)
        valor_final=str(valor_final)
        fecha_inicio=date.strftime(fecha_inicio, '%Y-%m-%d')
        fecha_fin=date.strftime(fecha_fin, '%Y-%m-%d')
        if type(departamento)==str:
            departamento=[departamento]
        if type(anio)==str:
            anio=[anio]
        if type(entidad)==str:
            entidad=[entidad]
        if type(unspsc)==str:
            unspsc=[unspsc]      
        lista_consulta=[]
        for dpto in departamento:
            for an in anio:
                an=int(an)
                for ent in entidad:
                    for u in unspsc:
                        try:
                            print(dpto,ent,fecha_inicio,fecha_fin,valor_inicial,valor_final,u)
                            df_dpto=consulta_mae(dpto,ent,fecha_inicio,fecha_fin,valor_inicial,valor_final,u)
                            lista_consulta.append(df_dpto)
                        except:
                            None
                            continue

        df=pd.concat(lista_consulta)
        df=df.dropna()
        df=df.drop_duplicates()
        table=dash_table.DataTable(
            id='table',
            columns=[{"name": i, "id": i} for i in df.columns],
            data=df.to_dict('records'),
            style_cell={'textAlign': 'left'},
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
        )
        card_1=dbc.Card(
            dbc.CardBody(
                [
                    html.H4("Número de contratos", className="card-title"),
                    html.P(
                        df['cantidad'].sum(),
                        className="card-text",
                    ),
                ]
            ),
            style={"width": "18rem"},
        )
        card_2=dbc.Card(
            dbc.CardBody(
                [
                    html.H4("Valor total de contratos", className="card-title"),
                    html.P(
                        df['suma_valor_contrato'].sum(),
                        className="card-text",
                    ),
                ]
            ),
            style={"width": "18rem"},
        )
        card_3=dbc.Card(
            dbc.CardBody(
                [
                    html.H4("Promedio valor de Contratos", className="card-title"),
                    html.P(
                        df['suma_valor_contrato'].sum()/df['cantidad'].sum(),
                        className="card-text",
                    ),
                ]
            ),
            style={"width": "18rem"},
        ),
        fig1=px.bar(df, x="año", y="suma_valor_contrato", color="departamento_entidad")
        fig2=px.bar(df, x="año", y="cantidad",  color="departamento_entidad")
        return card_1,card_2,card_3,fig1,fig2#,table

    return app.run_server(mode='inline',debug=True,port=port)

In [6]:
consulta_MAE()

In [23]:
consulta_datos_departamento('Cundinamarca',2022)

,entidad,nit_entidad,año,orden,departamento_entidad,cantidad,suma_valor_contrato
391,ALCALDIA DE GAMA,800094684,2022,TERRITORIAL,Cundinamarca,6,3.238933e+08
427,ALCALDIA DE TABIO,899999443,2022,TERRITORIAL,Cundinamarca,6,2.942928e+08
429,ALCALDIA DE VERGARA CUNDINAMARCA,899999448,2022,NACIONAL,Cundinamarca,1,2.921365e+08
543,ALCALDIA MUNICIPAL ALBAN,899999450,2022,NACIONAL,Cundinamarca,2,2.641110e+08
583,ALCALDIA MUNICIPAL DE CHIA,899999172,2022,TERRITORIAL,Cundinamarca,804,1.039647e+11
...,...,...,...,...,...,...,...
70835,SENA REGIONAL CUNDINAMARCA Grupo Administrativ...,899999034,2022,NACIONAL,Cundinamarca,351,1.333132e+10
70838,SENA REGIONAL CUNDINAMARCA Grupo Administrativ...,899999034,2022,NACIONAL,Cundinamarca,387,1.472935e+10
70841,SENA REGIONAL CUNDINAMARCA Grupo Administrativ...,899999034,2022,NACIONAL,Cundinamarca,292,9.784916e+09
70844,SENA REGIONAL CUNDINAMARCA Grupo Administrativ...,899999034,2022,NACIONAL,Cundinamarca,282,1.053682e+10
